# CLEANING DATA FILE

### In this file "Cleaning_data," there will be an exploration of the data, a decision will be made whether to remove columns that are not useful for the analysis, duplicates and null values will be removed, tables will be merged, and new columns will be created with the goal of being used later in the analysis.

# IMPORTS

In [138]:
import pandas as pd
import os
import numpy as np

## CONCAT TRAIN DATA

### Data from 1999 to 2017

### Demographic data

In [139]:
folder_path = "./data/raw_data/DEMO"

# Lista para guardar los nombres de las variables creadas
variable_names = []

# Recorrer archivos
for filename in os.listdir(folder_path):
    # Saltar archivos que contienen "L" o "P" (antes de la extensión .csv)
    if filename.endswith(".csv") and "L" not in filename and "P" not in filename:
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)


Variable creada: DEMO_1999
Variable creada: DEMO_B_2001
Variable creada: DEMO_C_2003
Variable creada: DEMO_D_2005
Variable creada: DEMO_E_2007
Variable creada: DEMO_F_2009
Variable creada: DEMO_G_2011
Variable creada: DEMO_H_2013
Variable creada: DEMO_I_2015
Variable creada: DEMO_J_2017

Todas las variables creadas:
['DEMO_1999', 'DEMO_B_2001', 'DEMO_C_2003', 'DEMO_D_2005', 'DEMO_E_2007', 'DEMO_F_2009', 'DEMO_G_2011', 'DEMO_H_2013', 'DEMO_I_2015', 'DEMO_J_2017']


In [140]:
# Columnas que quieres conservar (las que estén disponibles)
cols_deseadas = ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN', 'INDFMPIR', 'DMDBORN4','DMDBORN2']

for name in variable_names:
    df = globals()[name]
    
    # Filtrar columnas que existen en este DataFrame
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    # Si al menos hay una columna deseada, seguimos
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Diccionario de renombrado
rename_dict = {
    'RIAGENDR': 'Gender',
    'RIDAGEYR': 'Age(year)',
    'RIDRETH3': 'Race',
    'RIDRETH1': 'Race',  # si el dataset usa RIDRETH1 en vez de RIDRETH3
    'DMDBORN4': 'Country_of_birth',
    'DMDBORN': 'Country_of_birth',
    'DMDBORN2': 'Country_of_birth',  # por si usa DMDBORN
    'INDFMPIR': 'Income_poverty_num'
}

# Renombrar columnas en todos los DataFrames
for name in variable_names:
    df = globals()[name]
    # Solo renombra si las columnas existen
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df  # actualizar la variable
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print ("\n")
# Asegurarnos de que todos los DataFrames estén ordenados por SEQN
for name in variable_names:
    df = globals()[name]
    if 'SEQN' in df.columns:
        df.sort_values('SEQN', inplace=True)
        df.reset_index(drop=True, inplace=True)
        globals()[name] = df

# Concatenar todos por columnas
dfs_to_concat = [globals()[name] for name in variable_names]
train_demographic = pd.concat(dfs_to_concat, axis=0)
train_demographic['Country_of_birth'] = train_demographic['Country_of_birth'].replace({3.0: 2.0, 4.0: 2.0, 5.0:2.0, 77.0:np.nan})

def categorize_income(x):
    if x < 1.0:
        return 1.0 # below_poverty
    elif x < 2.0:
        return 2.0 #low_income
    elif x < 3.0:
        return 3.0 #'middle_income'
    elif x < 5.0:
        return 4.0 #'high_income'
    elif x == 5.0:
        return 5 #'very_high_income'
    else:
        return np.nan
    
train_demographic['Income_category'] = train_demographic['Income_poverty_num'].apply(categorize_income)
train_demographic.drop(columns=['Income_poverty_num'], inplace=True)

train_demographic.head()

DEMO_1999: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN', 'INDFMPIR']
DEMO_B_2001: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN', 'INDFMPIR']
DEMO_C_2003: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN', 'INDFMPIR']
DEMO_D_2005: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN', 'INDFMPIR']
DEMO_E_2007: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMPIR', 'DMDBORN2']
DEMO_F_2009: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMPIR', 'DMDBORN2']
DEMO_G_2011: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMPIR', 'DMDBORN4']
DEMO_H_2013: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMPIR', 'DMDBORN4']
DEMO_I_2015: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMPIR', 'DMDBORN4']
DEMO_J_2017: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 

,SEQN,Gender,Age(year),Race,Country_of_birth,Income_category
0,1.0,2.0,2.0,4.0,1.0,1.0
1,2.0,1.0,77.0,3.0,1.0,5.0
2,3.0,2.0,10.0,3.0,2.0,2.0
3,4.0,1.0,1.0,4.0,1.0,1.0
4,5.0,1.0,49.0,3.0,1.0,5.0


In [141]:
train_demographic.dropna(inplace=True)
train_demographic.info()
train_demographic.to_csv('./data/Train_data/train_demographic.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 92112 entries, 0 to 9253
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEQN              92112 non-null  float64
 1   Gender            92112 non-null  float64
 2   Age(year)         92112 non-null  float64
 3   Race              92112 non-null  float64
 4   Country_of_birth  92112 non-null  float64
 5   Income_category   92112 non-null  float64
dtypes: float64(6)
memory usage: 4.9 MB


### Weight and Height train data

In [142]:
folder_path = "./data/raw_data/weight"

# Lista para guardar los nombres de las variables creadas
variable_names = []

# Recorrer archivos
for filename in os.listdir(folder_path):
    # Saltar archivos que contienen "L" o "P" (antes de la extensión .csv)
    if filename.endswith(".csv") and "_L_" not in filename and "P_" not in filename:
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)



Variable creada: WHQ_1999
Variable creada: WHQ_B_2001
Variable creada: WHQ_C_2003
Variable creada: WHQ_D_2005
Variable creada: WHQ_E_2007
Variable creada: WHQ_F_2009
Variable creada: WHQ_G_2011
Variable creada: WHQ_H_2013
Variable creada: WHQ_I_2015
Variable creada: WHQ_J_2017

Todas las variables creadas:
['WHQ_1999', 'WHQ_B_2001', 'WHQ_C_2003', 'WHQ_D_2005', 'WHQ_E_2007', 'WHQ_F_2009', 'WHQ_G_2011', 'WHQ_H_2013', 'WHQ_I_2015', 'WHQ_J_2017']


In [143]:
# Columnas que quieres conservar (las que estén disponibles)
cols_deseadas = ["SEQN", "WHD010", "WHD020"]

for name in variable_names:
    df = globals()[name]
    
    # Filtrar columnas que existen en este DataFrame
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Diccionario de renombrado
rename_dict = {
    'WHD010': 'Height(m)',    # inicialmente en pulgadas
    'WHD020': 'Weight(kg)'    # inicialmente en libras
}

# Renombrar columnas en todos los DataFrames
for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Ordenar todos los DataFrames por SEQN
for name in variable_names:
    df = globals()[name]
    if 'SEQN' in df.columns:
        df.sort_values('SEQN', inplace=True)
        df.reset_index(drop=True, inplace=True)
        globals()[name] = df

# Concatenar por filas (uno debajo de otro)
dfs_to_concat = [globals()[name] for name in variable_names]
df_weight_train = pd.concat(dfs_to_concat, axis=0, ignore_index=True)

df_weight_train['Height(m)'] = df_weight_train['Height(m)'].replace({99999.0: np.nan, 77777.0: np.nan, 9999.0: np.nan, 7777.0: np.nan})
df_weight_train['Weight(kg)'] = df_weight_train['Weight(kg)'].replace({99999.0: np.nan, 77777.0: np.nan, 9999.0: np.nan, 7777.0: np.nan})

# ✅ Convertir unidades
# WHD010 era en pulgadas → Height(m)
# WHD020 era en libras → Weight(kg)
df_weight_train['Height(m)'] = df_weight_train['Height(m)'] * 0.0254
df_weight_train['Weight(kg)'] = df_weight_train['Weight(kg)'] * 0.453592

# Mostrar resultado final
print("\nDataFrame final con unidades convertidas:")
print(df_weight_train.head())

# Guardar (opcional)
df_weight_train.head()

WHQ_1999: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_B_2001: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_C_2003: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_D_2005: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_E_2007: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_F_2009: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_G_2011: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_H_2013: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_I_2015: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_J_2017: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']


WHQ_1999: columnas renombradas → ['Height(m)', 'Weight(kg)']
WHQ_B_2001: columnas renombradas → ['Height(m)', 'Weight(kg)']
WHQ_C_2003: columnas renombradas → ['Height(m)', 'Weight(kg)']
WHQ_D_2005: columnas renombradas → ['Height(m)', 'Weight(kg)']
WHQ_E_2007: columnas renombradas → ['Height(m)', 'Weight(kg)']
WHQ_F_2009: columnas renombradas → ['Height(m)', 'Weight(kg)']
WHQ_

,SEQN,Height(m),Weight(kg)
0,2.0,1.7526,76.203456
1,5.0,1.8034,93.439952
2,6.0,1.6256,59.874144
3,7.0,1.6510,79.832192
4,10.0,1.9050,90.718400


In [144]:

df_weight_train.dropna(subset=['Height(m)', 'Weight(kg)'], inplace=True)


df_weight_train.info()


df_weight_train.to_csv('./data/Train_data/train_weight.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 61296 entries, 0 to 63591
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEQN        61296 non-null  float64
 1   Height(m)   61296 non-null  float64
 2   Weight(kg)  61296 non-null  float64
dtypes: float64(3)
memory usage: 1.9 MB


In [145]:
df_weight_train['Weight(kg)'].max()


303.90664

### HDL_Train

In [146]:
folder_path = "./data/raw_data/HDL"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and not ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)


Variable creada: HDL_D_2005
Variable creada: HDL_E_2007
Variable creada: HDL_F_2009
Variable creada: HDL_G_2011
Variable creada: HDL_H_2013
Variable creada: HDL_I_2015
Variable creada: HDL_J_2017
Variable creada: L13_B_2001
Variable creada: L13_C_2003
Variable creada: LAB13_1999

Todas las variables creadas:
['HDL_D_2005', 'HDL_E_2007', 'HDL_F_2009', 'HDL_G_2011', 'HDL_H_2013', 'HDL_I_2015', 'HDL_J_2017', 'L13_B_2001', 'L13_C_2003', 'LAB13_1999']


In [147]:
folder_path = "./data/raw_data/chol_total"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and not ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)


Variable creada: TCHOL_D_2005
Variable creada: TCHOL_E_2007
Variable creada: TCHOL_F_2009
Variable creada: TCHOL_G_2011
Variable creada: TCHOL_H_2013
Variable creada: TCHOL_I_2015
Variable creada: TCHOL_J_2017

Todas las variables creadas:
['HDL_D_2005', 'HDL_E_2007', 'HDL_F_2009', 'HDL_G_2011', 'HDL_H_2013', 'HDL_I_2015', 'HDL_J_2017', 'L13_B_2001', 'L13_C_2003', 'LAB13_1999', 'TCHOL_D_2005', 'TCHOL_E_2007', 'TCHOL_F_2009', 'TCHOL_G_2011', 'TCHOL_H_2013', 'TCHOL_I_2015', 'TCHOL_J_2017']


In [148]:
import pandas as pd

# Columnas a conservar
cols_deseadas = ['SEQN', 'LBDHDDSI', 'LBDHDLSI', 'LBDTCSI']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas (si existen)
rename_dict = {
    'LBDHDDSI': 'HDL-Cholesterol(mmol/L)',
    'LBDHDLSI': 'HDL-Cholesterol(mmol/L)',  # Algunos archivos usan esta
    'LBDTCSI': 'Total_Cholesterol(mmol/L)'
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Merge año por año entre HDL y TCHOL
merged_dfs = []
years = ['2005', '2007', '2009', '2011', '2013', '2015', '2017']

for year in years:
    # Buscar variables que contengan el año
    hdl_vars = [v for v in variable_names if year in v and v.startswith("HDL")]
    tchol_vars = [v for v in variable_names if year in v and v.startswith("TCHOL")]

    if hdl_vars and tchol_vars:
        hdl_var = hdl_vars[0]
        tchol_var = tchol_vars[0]

        df_hdl = globals()[hdl_var]
        df_tchol = globals()[tchol_var]

        # Merge por SEQN
        df_merged = pd.merge(df_hdl, df_tchol, on='SEQN', how='outer')
        df_merged['Year'] = year
        merged_dfs.append(df_merged)

        print(f"✅ {year}: Merge completado entre {hdl_var} y {tchol_var}")
    else:
        print(f"⚠️ {year}: No se encontró HDL o TCHOL correspondiente.")

# Concatenar todos los merges
df_HDL_TCHOL_train = pd.concat(merged_dfs, axis=0, ignore_index=True)
df_HDL_TCHOL_train.drop(columns="Year",inplace=True)

# Resultado final
print("\n🔍 DataFrame final HDL + TCHOL:")
df_HDL_TCHOL_train.head()




HDL_D_2005: columnas filtradas → ['SEQN', 'LBDHDDSI']
HDL_E_2007: columnas filtradas → ['SEQN', 'LBDHDDSI']
HDL_F_2009: columnas filtradas → ['SEQN', 'LBDHDDSI']
HDL_G_2011: columnas filtradas → ['SEQN', 'LBDHDDSI']
HDL_H_2013: columnas filtradas → ['SEQN', 'LBDHDDSI']
HDL_I_2015: columnas filtradas → ['SEQN', 'LBDHDDSI']
HDL_J_2017: columnas filtradas → ['SEQN', 'LBDHDDSI']
L13_B_2001: columnas filtradas → ['SEQN', 'LBDHDLSI', 'LBDTCSI']
L13_C_2003: columnas filtradas → ['SEQN', 'LBDHDDSI', 'LBDTCSI']
LAB13_1999: columnas filtradas → ['SEQN', 'LBDHDLSI', 'LBDTCSI']
TCHOL_D_2005: columnas filtradas → ['SEQN', 'LBDTCSI']
TCHOL_E_2007: columnas filtradas → ['SEQN', 'LBDTCSI']
TCHOL_F_2009: columnas filtradas → ['SEQN', 'LBDTCSI']
TCHOL_G_2011: columnas filtradas → ['SEQN', 'LBDTCSI']
TCHOL_H_2013: columnas filtradas → ['SEQN', 'LBDTCSI']
TCHOL_I_2015: columnas filtradas → ['SEQN', 'LBDTCSI']
TCHOL_J_2017: columnas filtradas → ['SEQN', 'LBDTCSI']


HDL_D_2005: columnas renombradas → ['HDL

,SEQN,HDL-Cholesterol(mmol/L),Total_Cholesterol(mmol/L)
0,31128.0,1.42,3.34
1,31129.0,1.19,4.40
2,31130.0,NaN,NaN
3,31131.0,1.01,2.72
4,31132.0,1.53,3.80


In [149]:
df_HDL_TCHOL_train.dropna(inplace=True)
df_HDL_TCHOL_train.info()
df_HDL_TCHOL_train.to_csv('./data/Train_data/train_HDL_TCHOL.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 51199 entries, 0 to 56376
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQN                       51199 non-null  float64
 1   HDL-Cholesterol(mmol/L)    51199 non-null  float64
 2   Total_Cholesterol(mmol/L)  51199 non-null  float64
dtypes: float64(3)
memory usage: 1.6 MB


### Glycohemoglobin

In [150]:
folder_path = "./data/raw_data/GHB"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and not ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: GHB_D_2005
Variable creada: GHB_E_2007
Variable creada: GHB_F_2009
Variable creada: GHB_G_2011
Variable creada: GHB_H_2013
Variable creada: GHB_I_2015
Variable creada: GHB_J_2017
Variable creada: L10_B_2001
Variable creada: L10_C_2003
Variable creada: LAB10_1999

Todas las variables creadas:
['GHB_D_2005', 'GHB_E_2007', 'GHB_F_2009', 'GHB_G_2011', 'GHB_H_2013', 'GHB_I_2015', 'GHB_J_2017', 'L10_B_2001', 'L10_C_2003', 'LAB10_1999']


In [151]:
# Columnas que quieres conservar (las que estén disponibles)
cols_deseadas = ["SEQN",'LBXGH']

for name in variable_names:
    df = globals()[name]
    
    # Filtrar columnas que existen en este DataFrame
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Diccionario de renombrado
rename_dict = {
    'LBXGH': 'Glycohemoglobin(%)'
}

# Renombrar columnas en todos los DataFrames
for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Ordenar todos los DataFrames por SEQN
for name in variable_names:
    df = globals()[name]
    if 'SEQN' in df.columns:
        df.sort_values('SEQN', inplace=True)
        df.reset_index(drop=True, inplace=True)
        globals()[name] = df

# Concatenar por filas (uno debajo de otro)
dfs_to_concat = [globals()[name] for name in variable_names]
df_GHB_train = pd.concat(dfs_to_concat, axis=0, ignore_index=True)


# Mostrar resultado final
print("\nDataFrame final con unidades convertidas:")
print(df_GHB_train.head())

# Guardar (opcional)
df_GHB_train.head()

GHB_D_2005: columnas filtradas → ['SEQN', 'LBXGH']
GHB_E_2007: columnas filtradas → ['SEQN', 'LBXGH']
GHB_F_2009: columnas filtradas → ['SEQN', 'LBXGH']
GHB_G_2011: columnas filtradas → ['SEQN', 'LBXGH']
GHB_H_2013: columnas filtradas → ['SEQN', 'LBXGH']
GHB_I_2015: columnas filtradas → ['SEQN', 'LBXGH']
GHB_J_2017: columnas filtradas → ['SEQN', 'LBXGH']
L10_B_2001: columnas filtradas → ['SEQN', 'LBXGH']
L10_C_2003: columnas filtradas → ['SEQN', 'LBXGH']
LAB10_1999: columnas filtradas → ['SEQN', 'LBXGH']


GHB_D_2005: columnas renombradas → ['Glycohemoglobin(%)']
GHB_E_2007: columnas renombradas → ['Glycohemoglobin(%)']
GHB_F_2009: columnas renombradas → ['Glycohemoglobin(%)']
GHB_G_2011: columnas renombradas → ['Glycohemoglobin(%)']
GHB_H_2013: columnas renombradas → ['Glycohemoglobin(%)']
GHB_I_2015: columnas renombradas → ['Glycohemoglobin(%)']
GHB_J_2017: columnas renombradas → ['Glycohemoglobin(%)']
L10_B_2001: columnas renombradas → ['Glycohemoglobin(%)']
L10_C_2003: columnas ren

,SEQN,Glycohemoglobin(%)
0,31129.0,5.2
1,31130.0,NaN
2,31131.0,6.0
3,31132.0,7.1
4,31133.0,4.7


In [152]:
df_GHB_train.dropna(inplace=True)
df_GHB_train.info()
df_GHB_train.to_csv('./data/Train_data/train_GHB.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 64952 entries, 0 to 69131
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SEQN                64952 non-null  float64
 1   Glycohemoglobin(%)  64952 non-null  float64
dtypes: float64(2)
memory usage: 1.5 MB


### Insulin

In [153]:
folder_path = "./data/raw_data/INS"

# Lista para guardar los nombres de las variables creadas
variable_names = []

# Recorrer archivos
for filename in os.listdir(folder_path):
    # Saltar archivos que contienen "L" o "P" (antes de la extensión .csv)
    if filename.endswith(".csv") and "_L_" not in filename and "P_" not in filename:
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)


Variable creada: INS_H_2013
Variable creada: INS_I_2015
Variable creada: INS_J_2017
Variable creada: L10AM_C_2003
Variable creada: L10_2_B_2001
Variable creada: LAB10AM_1999

Todas las variables creadas:
['INS_H_2013', 'INS_I_2015', 'INS_J_2017', 'L10AM_C_2003', 'L10_2_B_2001', 'LAB10AM_1999']


In [154]:

folder_path = "./data/raw_data/GLU"
# Años que quieres filtrar
years = ["2005", "2007", "2009", "2011"]

# Recorrer archivos
for filename in os.listdir(folder_path):
    # Verificar que sea CSV y que contenga alguno de los años deseados
    if filename.endswith(".csv") and any(year in filename for year in years):
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)



Variable creada: GLU_D_2005
Variable creada: GLU_E_2007
Variable creada: GLU_F_2009
Variable creada: GLU_G_2011

Todas las variables creadas:
['INS_H_2013', 'INS_I_2015', 'INS_J_2017', 'L10AM_C_2003', 'L10_2_B_2001', 'LAB10AM_1999', 'GLU_D_2005', 'GLU_E_2007', 'GLU_F_2009', 'GLU_G_2011']


In [155]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'LBDINSI', 'LB2INSI', 'LBXINSI']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'LBDINSI': 'Insulin (pmol/L)',
    'LB2INSI': 'Insulin (pmol/L)',
    'LBXINSI': 'Insulin (pmol/L)'
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if 'Insulin (pmol/L)' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_INS_Train = pd.concat(merged_dfs, axis=0, ignore_index=True)

# Resultado final
print("\n🔍 DataFrame final de insulina:")
df_INS_Train.head()


INS_H_2013: columnas filtradas → ['SEQN', 'LBDINSI']
INS_I_2015: columnas filtradas → ['SEQN', 'LBDINSI']
INS_J_2017: columnas filtradas → ['SEQN', 'LBDINSI']
L10AM_C_2003: columnas filtradas → ['SEQN', 'LBDINSI']
L10_2_B_2001: columnas filtradas → ['SEQN', 'LB2INSI']
LAB10AM_1999: columnas filtradas → ['SEQN', 'LBXINSI']
GLU_D_2005: columnas filtradas → ['SEQN', 'LBDINSI']
GLU_E_2007: columnas filtradas → ['SEQN', 'LBDINSI']
GLU_F_2009: columnas filtradas → ['SEQN', 'LBDINSI']
GLU_G_2011: columnas filtradas → ['SEQN', 'LBDINSI']


INS_H_2013: columnas renombradas → ['Insulin (pmol/L)']
INS_I_2015: columnas renombradas → ['Insulin (pmol/L)']
INS_J_2017: columnas renombradas → ['Insulin (pmol/L)']
L10AM_C_2003: columnas renombradas → ['Insulin (pmol/L)']
L10_2_B_2001: columnas renombradas → ['Insulin (pmol/L)']
LAB10AM_1999: columnas renombradas → ['Insulin (pmol/L)']
GLU_D_2005: columnas renombradas → ['Insulin (pmol/L)']
GLU_E_2007: columnas renombradas → ['Insulin (pmol/L)']
GLU_F_20

,SEQN,Insulin (pmol/L)
0,73559.0,34.98
1,73561.0,36.72
2,73564.0,89.46
3,73568.0,23.10
4,73574.0,36.30


In [156]:
df_INS_Train.dropna(inplace=True)
df_INS_Train.info()
df_INS_Train.to_csv('./data/Train_data/train_INS.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 27535 entries, 0 to 30221
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEQN              27535 non-null  float64
 1   Insulin (pmol/L)  27535 non-null  float64
dtypes: float64(2)
memory usage: 645.4 KB


### BIOPROFILE Train

In [157]:
folder_path = "./data/raw_data/BIOPRO"

# Lista para guardar los nombres de las variables creadas
variable_names = []

# Recorrer archivos
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: BIOPRO_D_2005
Variable creada: BIOPRO_E_2007
Variable creada: BIOPRO_F_2009
Variable creada: BIOPRO_G_2011
Variable creada: BIOPRO_H_2013
Variable creada: BIOPRO_I_2015
Variable creada: BIOPRO_J_2017
Variable creada: L40_B_2001
Variable creada: L40_C_2003
Variable creada: LAB18_1999
Variable creada: P_BIOPRO_2017

Todas las variables creadas:
['BIOPRO_D_2005', 'BIOPRO_E_2007', 'BIOPRO_F_2009', 'BIOPRO_G_2011', 'BIOPRO_H_2013', 'BIOPRO_I_2015', 'BIOPRO_J_2017', 'L40_B_2001', 'L40_C_2003', 'LAB18_1999', 'P_BIOPRO_2017']


In [158]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'LBDSGLSI', 'LBDSCRSI','LBDSUASI','LBDSCASI','LBDSTPSI','LBDSGBSI']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'LBDSGLSI': 'Glucose in serum (nmol/L)',
    'LBDSCRSI': 'Creatinine (umol/L)',
    'LBDSUASI': 'Uric acid (umol/L)',
    'LBDSCASI': 'Calcium (mmol/L)',
    'LBDSTPSI': 'Total protein (g/L)',
    'LBDSGBSI': 'Globulin (g/L)'
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if 'Glucose in serum (nmol/L)' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_bioprofile_train = pd.concat( merged_dfs, axis=0, ignore_index=True)

# Resultado final
print("\n🔍 DataFrame final de insulina:")
print(df_bioprofile_train.head())


BIOPRO_D_2005: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
BIOPRO_E_2007: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
BIOPRO_F_2009: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
BIOPRO_G_2011: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
BIOPRO_H_2013: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
BIOPRO_I_2015: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
BIOPRO_J_2017: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
L40_B_2001: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', 'LBDSUASI', 'LBDSCASI', 'LBDSTPSI', 'LBDSGBSI']
L40_C_2003: columnas filtradas → ['SEQN', 'LBDSGLSI', 'LBDSCRSI', '

In [159]:
df_bioprofile_train.dropna(inplace=True)
df_bioprofile_train.info()
df_bioprofile_train.to_csv('./data/Train_data/train_bioprofile.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 73326 entries, 0 to 79540
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQN                       73326 non-null  float64
 1   Glucose in serum (nmol/L)  73326 non-null  float64
 2   Creatinine (umol/L)        73326 non-null  float64
 3   Uric acid (umol/L)         73326 non-null  float64
 4   Calcium (mmol/L)           73326 non-null  float64
 5   Total protein (g/L)        73326 non-null  float64
 6   Globulin (g/L)             73326 non-null  float64
dtypes: float64(7)
memory usage: 4.5 MB


### Blood plessure

In [160]:
folder_path = "./data/raw_data/BPQ"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and not ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: BPQ_1999
Variable creada: BPQ_B_2001
Variable creada: BPQ_C_2003
Variable creada: BPQ_D_2005
Variable creada: BPQ_E_2007
Variable creada: BPQ_F_2009
Variable creada: BPQ_G_2011
Variable creada: BPQ_H_2013
Variable creada: BPQ_I_2015
Variable creada: BPQ_J_2017

Todas las variables creadas:
['BPQ_1999', 'BPQ_B_2001', 'BPQ_C_2003', 'BPQ_D_2005', 'BPQ_E_2007', 'BPQ_F_2009', 'BPQ_G_2011', 'BPQ_H_2013', 'BPQ_I_2015', 'BPQ_J_2017']


In [161]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'BPQ020']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'BPQ020': 'High_blood_pressure?'
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if 'High_blood_pressure?' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_blood_train = pd.concat( merged_dfs, axis=0, ignore_index=True)

df_blood_train['High_blood_pressure?'] = df_blood_train['High_blood_pressure?'].replace({9.0: np.nan})


# Resultado final
print("\n🔍 DataFrame final de insulina:")
df_blood_train.head()

BPQ_1999: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_B_2001: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_C_2003: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_D_2005: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_E_2007: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_F_2009: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_G_2011: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_H_2013: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_I_2015: columnas filtradas → ['SEQN', 'BPQ020']
BPQ_J_2017: columnas filtradas → ['SEQN', 'BPQ020']


BPQ_1999: columnas renombradas → ['High_blood_pressure?']
BPQ_B_2001: columnas renombradas → ['High_blood_pressure?']
BPQ_C_2003: columnas renombradas → ['High_blood_pressure?']
BPQ_D_2005: columnas renombradas → ['High_blood_pressure?']
BPQ_E_2007: columnas renombradas → ['High_blood_pressure?']
BPQ_F_2009: columnas renombradas → ['High_blood_pressure?']
BPQ_G_2011: columnas renombradas → ['High_blood_pressure?']
BPQ_H_2013: columnas renombradas → ['High_blood_pressure?']
BP

,SEQN,High_blood_pressure?
0,2.0,2.0
1,5.0,1.0
2,6.0,2.0
3,7.0,1.0
4,10.0,2.0


In [162]:
df_blood_train.dropna(inplace=True)
df_blood_train.info()
df_blood_train.to_csv('./data/Train_data/train_blood.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 63219 entries, 0 to 63591
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SEQN                  63219 non-null  float64
 1   High_blood_pressure?  63219 non-null  float64
dtypes: float64(2)
memory usage: 1.4 MB


### Alcohol use

In [163]:
folder_path = "./data/raw_data/ALQ"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and not ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: ALQ_1999
Variable creada: ALQ_B_2001
Variable creada: ALQ_C_2003
Variable creada: ALQ_D_2005
Variable creada: ALQ_E_2007
Variable creada: ALQ_F_2009
Variable creada: ALQ_G_2011
Variable creada: ALQ_H_2013
Variable creada: ALQ_I_2015
Variable creada: ALQ_J_2017

Todas las variables creadas:
['ALQ_1999', 'ALQ_B_2001', 'ALQ_C_2003', 'ALQ_D_2005', 'ALQ_E_2007', 'ALQ_F_2009', 'ALQ_G_2011', 'ALQ_H_2013', 'ALQ_I_2015', 'ALQ_J_2017']


### Smoking use

In [164]:
folder_path = "./data/raw_data/SMQ"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and not ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: SMQ_1999
Variable creada: SMQ_B_2001
Variable creada: SMQ_C_2003
Variable creada: SMQ_D_2005
Variable creada: SMQ_E_2007
Variable creada: SMQ_F_2009
Variable creada: SMQ_G_2011
Variable creada: SMQ_H_2013
Variable creada: SMQ_I_2015
Variable creada: SMQ_J_2017

Todas las variables creadas:
['SMQ_1999', 'SMQ_B_2001', 'SMQ_C_2003', 'SMQ_D_2005', 'SMQ_E_2007', 'SMQ_F_2009', 'SMQ_G_2011', 'SMQ_H_2013', 'SMQ_I_2015', 'SMQ_J_2017']


In [165]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'SMQ020']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'SMQ020': '100_cigarrettes_life?'
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if '100_cigarrettes_life?' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_smoking_train = pd.concat( merged_dfs, axis=0, ignore_index=True)

df_smoking_train['100_cigarrettes_life?'] = df_smoking_train['100_cigarrettes_life?'].replace({9.0: np.nan, 7.0: np.nan})


# Resultado final
print("\n🔍 DataFrame final de insulina:")
df_smoking_train.head()

SMQ_1999: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_B_2001: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_C_2003: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_D_2005: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_E_2007: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_F_2009: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_G_2011: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_H_2013: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_I_2015: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_J_2017: columnas filtradas → ['SEQN', 'SMQ020']


SMQ_1999: columnas renombradas → ['100_cigarrettes_life?']
SMQ_B_2001: columnas renombradas → ['100_cigarrettes_life?']
SMQ_C_2003: columnas renombradas → ['100_cigarrettes_life?']
SMQ_D_2005: columnas renombradas → ['100_cigarrettes_life?']
SMQ_E_2007: columnas renombradas → ['100_cigarrettes_life?']
SMQ_F_2009: columnas renombradas → ['100_cigarrettes_life?']
SMQ_G_2011: columnas renombradas → ['100_cigarrettes_life?']
SMQ_H_2013: columnas renombradas → ['100_cigarrettes_li

,SEQN,100_cigarrettes_life?
0,2.0,2.0
1,5.0,1.0
2,7.0,1.0
3,10.0,1.0
4,12.0,2.0


In [166]:
df_smoking_train.dropna(inplace=True)
df_smoking_train.info()
df_smoking_train.to_csv('./data/Train_data/train_smoking.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 55918 entries, 0 to 64873
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SEQN                   55918 non-null  float64
 1   100_cigarrettes_life?  55918 non-null  float64
dtypes: float64(2)
memory usage: 1.3 MB


### Diabetes (target) train data

In [167]:
folder_path = "./data/raw_data/diabetes"

# Lista para guardar los nombres de las variables creadas
variable_names = []

# Recorrer archivos
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: DIQ_1999
Variable creada: DIQ_B_2001
Variable creada: DIQ_C_2003
Variable creada: DIQ_D_2005
Variable creada: DIQ_E_2007
Variable creada: DIQ_F_2009
Variable creada: DIQ_G_2011
Variable creada: DIQ_H_2013
Variable creada: DIQ_I_2015
Variable creada: DIQ_J_2017
Variable creada: DIQ_L_2021
Variable creada: P_DIQ_2017

Todas las variables creadas:
['DIQ_1999', 'DIQ_B_2001', 'DIQ_C_2003', 'DIQ_D_2005', 'DIQ_E_2007', 'DIQ_F_2009', 'DIQ_G_2011', 'DIQ_H_2013', 'DIQ_I_2015', 'DIQ_J_2017', 'DIQ_L_2021', 'P_DIQ_2017']


In [168]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'DIQ010']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'DIQ010': 'Diabetes_diagnosticated',
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if 'Diabetes_diagnosticated' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_diabetes_train = pd.concat( merged_dfs, axis=0, ignore_index=True)

# Resultado final
print("\n🔍 DataFrame final de insulina:")
print(df_diabetes_train.head())


DIQ_1999: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_B_2001: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_C_2003: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_D_2005: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_E_2007: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_F_2009: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_G_2011: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_H_2013: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_I_2015: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_J_2017: columnas filtradas → ['SEQN', 'DIQ010']
DIQ_L_2021: columnas filtradas → ['SEQN', 'DIQ010']
P_DIQ_2017: columnas filtradas → ['SEQN', 'DIQ010']


DIQ_1999: columnas renombradas → ['Diabetes_diagnosticated']
DIQ_B_2001: columnas renombradas → ['Diabetes_diagnosticated']
DIQ_C_2003: columnas renombradas → ['Diabetes_diagnosticated']
DIQ_D_2005: columnas renombradas → ['Diabetes_diagnosticated']
DIQ_E_2007: columnas renombradas → ['Diabetes_diagnosticated']
DIQ_F_2009: columnas renombradas → ['Diabetes_diagnosticated']


In [169]:
df_diabetes_train['Diabetes_diagnosticated'] = df_diabetes_train['Diabetes_diagnosticated'].map({
    1.0: 1,  # Sí
    2.0: 0,  # No
    3.0: 1.0, # Limítrofe → se incluye como positivo
    9.0: np.nan  # No sabe
})


df_diabetes_train.dropna(inplace=True)
df_diabetes_train.info()
df_diabetes_train.to_csv('./data/Train_data/train_diabetes.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 123459 entries, 0 to 123540
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   SEQN                     123459 non-null  float64
 1   Diabetes_diagnosticated  123459 non-null  float64
dtypes: float64(2)
memory usage: 2.8 MB


### MERGE ALL DATAFRAMES TO TRAIN

In [170]:
df_final_train = pd.merge(train_demographic, df_weight_train, on="SEQN", how="inner")
df_final_train = pd.merge(df_final_train, df_HDL_TCHOL_train, on="SEQN", how="inner")
df_final_train = pd.merge(df_final_train, df_GHB_train, on="SEQN", how="inner")
df_final_train = pd.merge(df_final_train, df_INS_Train, on="SEQN", how="inner")
df_final_train = pd.merge(df_final_train, df_blood_train, on="SEQN", how="inner") 
df_final_train = pd.merge(df_final_train, df_smoking_train, on="SEQN", how="inner")  
# #df_final_train = pd.merge(df_final_train, df_bioprofile_train, on="SEQN", how="outer")
df_final_train = pd.merge(df_final_train, df_diabetes_train, on="SEQN", how="inner")
df_final_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15408 entries, 0 to 15407
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQN                       15408 non-null  float64
 1   Gender                     15408 non-null  float64
 2   Age(year)                  15408 non-null  float64
 3   Race                       15408 non-null  float64
 4   Country_of_birth           15408 non-null  float64
 5   Income_category            15408 non-null  float64
 6   Height(m)                  15408 non-null  float64
 7   Weight(kg)                 15408 non-null  float64
 8   HDL-Cholesterol(mmol/L)    15408 non-null  float64
 9   Total_Cholesterol(mmol/L)  15408 non-null  float64
 10  Glycohemoglobin(%)         15408 non-null  float64
 11  Insulin (pmol/L)           15408 non-null  float64
 12  High_blood_pressure?       15408 non-null  float64
 13  100_cigarrettes_life?      15408 non-null  flo

In [171]:
df_final_train.head()

,SEQN,Gender,Age(year),Race,Country_of_birth,Income_category,Height(m),Weight(kg),HDL-Cholesterol(mmol/L),Total_Cholesterol(mmol/L),Glycohemoglobin(%),Insulin (pmol/L),High_blood_pressure?,100_cigarrettes_life?,Diabetes_diagnosticated
0,31131.0,2.0,44.0,4.0,1.0,4.0,1.5240,74.389088,1.01,2.72,6.0,60.18,1.0,2.0,0.0
1,31132.0,1.0,70.0,3.0,1.0,5.0,1.7272,69.853168,1.53,3.80,7.1,53.94,1.0,2.0,1.0
2,31150.0,1.0,79.0,3.0,1.0,2.0,1.7526,86.182480,2.09,4.68,5.0,23.46,2.0,1.0,0.0
3,31151.0,2.0,59.0,4.0,1.0,4.0,1.7018,81.646560,1.97,5.30,5.8,36.18,1.0,1.0,1.0
4,31153.0,2.0,44.0,5.0,1.0,2.0,1.6764,104.326160,1.29,5.12,4.6,125.88,1.0,1.0,0.0


In [172]:
df_final_train.to_csv('./data/Train_data/train_data.csv', index=False)

# DATA TO PREDICT

### Demographic

In [173]:
folder_path = "./data/raw_data/DEMO"

# Lista para guardar los nombres de las variables creadas
variable_names = []

# Recorrer archivos
for filename in os.listdir(folder_path):
    # Cargar solo los archivos que contienen "L" o "P" (mayúscula o minúscula)
    if filename.endswith(".csv") and ("l" in filename.lower() or "p" in filename.lower()):
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)



Variable creada: DEMO_L_2021
Variable creada: P_DEMO_2017

Todas las variables creadas:
['DEMO_L_2021', 'P_DEMO_2017']


In [174]:
# Columnas que quieres conservar (las que estén disponibles)
cols_deseadas = ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN', 'INDFMPIR', 'DMDBORN4','DMDBORN2']

for name in variable_names:
    df = globals()[name]
    
    # Filtrar columnas que existen en este DataFrame
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    # Si al menos hay una columna deseada, seguimos
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Diccionario de renombrado
rename_dict = {
    'RIAGENDR': 'Gender',
    'RIDAGEYR': 'Age(year)',
    'RIDRETH3': 'Race',
    'RIDRETH1': 'Race',  # si el dataset usa RIDRETH1 en vez de RIDRETH3
    'DMDBORN4': 'Country_of_birth',
    'DMDBORN': 'Country_of_birth',
    'DMDBORN2': 'Country_of_birth',  # por si usa DMDBORN
    'INDFMPIR': 'Income_poverty_num'
}

# Renombrar columnas en todos los DataFrames
for name in variable_names:
    df = globals()[name]
    # Solo renombra si las columnas existen
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df  # actualizar la variable
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print ("\n")
# Asegurarnos de que todos los DataFrames estén ordenados por SEQN
for name in variable_names:
    df = globals()[name]
    if 'SEQN' in df.columns:
        df.sort_values('SEQN', inplace=True)
        df.reset_index(drop=True, inplace=True)
        globals()[name] = df

# Concatenar todos por columnas
dfs_to_concat = [globals()[name] for name in variable_names]
pre_demographic = pd.concat(dfs_to_concat, axis=0)
pre_demographic['Country_of_birth'] = pre_demographic['Country_of_birth'].replace({3.0: 2.0, 4.0: 2.0, 5.0:2.0, 77.0:np.nan})


def categorize_income(x):
    if x < 1.0:
        return 1.0 # below_poverty
    elif x < 2.0:
        return 2.0 #low_income
    elif x < 3.0:
        return 3.0 #'middle_income'
    elif x < 5.0:
        return 4.0 #'high_income'
    elif x == 5.0:
        return 5 #'very_high_income'
    else:
        return np.nan

pre_demographic['Income_category'] = pre_demographic['Income_poverty_num'].apply(categorize_income)
pre_demographic.drop(columns=['Income_poverty_num'], inplace=True)

pre_demographic.head()

DEMO_L_2021: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMPIR', 'DMDBORN4']
P_DEMO_2017: columnas filtradas → ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMPIR', 'DMDBORN4']


DEMO_L_2021: columnas renombradas → ['Gender', 'Age(year)', 'Race', 'Country_of_birth', 'Income_poverty_num']
P_DEMO_2017: columnas renombradas → ['Gender', 'Age(year)', 'Race', 'Country_of_birth', 'Income_poverty_num']




,SEQN,Gender,Age(year),Race,Country_of_birth,Income_category
0,130378.0,1.0,43.0,5.0,2.0,5.0
1,130379.0,1.0,66.0,3.0,1.0,5.0
2,130380.0,2.0,44.0,2.0,2.0,2.0
3,130381.0,2.0,5.0,5.0,1.0,2.0
4,130382.0,1.0,2.0,3.0,1.0,4.0


In [175]:
pre_demographic.dropna(inplace=True)
pre_demographic.info()
pre_demographic.to_csv('./data/Prep_data/pre_demographic.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 23248 entries, 0 to 15558
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEQN              23248 non-null  float64
 1   Gender            23248 non-null  float64
 2   Age(year)         23248 non-null  float64
 3   Race              23248 non-null  float64
 4   Country_of_birth  23248 non-null  float64
 5   Income_category   23248 non-null  float64
dtypes: float64(6)
memory usage: 1.2 MB


### weight 

In [176]:
folder_path = "./data/raw_data/weight"

# Lista para guardar los nombres de las variables creadas
variable_names = []

# Recorrer archivos
for filename in os.listdir(folder_path):
    # Cargar solo los archivos que contienen "L" o "P" (mayúscula o minúscula)
    if filename.endswith(".csv") and ("l" in filename.lower() or "p" in filename.lower()):
        file_path = os.path.join(folder_path, filename)

        # Crear nombre de variable seguro
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        
        # Leer el CSV y asignarlo como variable
        globals()[var_name] = pd.read_csv(file_path)

        # Guardar el nombre de la variable
        variable_names.append(var_name)
        
        print(f"Variable creada: {var_name}")

# Mostrar la lista resultante
print("\nTodas las variables creadas:")
print(variable_names)



Variable creada: P_WHQ_2017
Variable creada: WHQ_L_2021

Todas las variables creadas:
['P_WHQ_2017', 'WHQ_L_2021']


In [177]:
# Columnas que quieres conservar (las que estén disponibles)
cols_deseadas = ["SEQN", "WHD010", "WHD020"]

for name in variable_names:
    df = globals()[name]
    
    # Filtrar columnas que existen en este DataFrame
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Diccionario de renombrado
rename_dict = {
    'WHD010': 'Height(m)',    # inicialmente en pulgadas
    'WHD020': 'Weight(kg)'    # inicialmente en libras
}

# Renombrar columnas en todos los DataFrames
for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Ordenar todos los DataFrames por SEQN
for name in variable_names:
    df = globals()[name]
    if 'SEQN' in df.columns:
        df.sort_values('SEQN', inplace=True)
        df.reset_index(drop=True, inplace=True)
        globals()[name] = df

# Concatenar por filas (uno debajo de otro)
dfs_to_concat = [globals()[name] for name in variable_names]
df_weight_prep = pd.concat(dfs_to_concat, axis=0, ignore_index=True)

df_weight_prep['Height(m)'] = df_weight_prep['Height(m)'].replace({99999.0: np.nan, 77777.0: np.nan, 9999.0: np.nan, 7777.0: np.nan})
df_weight_prep['Weight(kg)'] = df_weight_prep['Weight(kg)'].replace({99999.0: np.nan, 77777.0: np.nan, 9999.0: np.nan, 7777.0: np.nan})


# ✅ Convertir unidades
# WHD010 era en pulgadas → Height(m)
# WHD020 era en libras → Weight(kg)
df_weight_prep['Height(m)'] = df_weight_prep['Height(m)'] * 0.0254
df_weight_prep['Weight(kg)'] = df_weight_prep['Weight(kg)'] * 0.453592

# Mostrar resultado final
print("\nDataFrame final con unidades convertidas:")
print(df_weight_prep.head())

# Guardar (opcional)
df_weight_prep.head()

P_WHQ_2017: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']
WHQ_L_2021: columnas filtradas → ['SEQN', 'WHD010', 'WHD020']


P_WHQ_2017: columnas renombradas → ['Height(m)', 'Weight(kg)']
WHQ_L_2021: columnas renombradas → ['Height(m)', 'Weight(kg)']



DataFrame final con unidades convertidas:
       SEQN  Height(m)  Weight(kg)
0  109266.0     1.6256   95.254320
1  109267.0     1.6002   58.966960
2  109268.0     1.6510   54.431040
3  109271.0     1.8288  100.697424
4  109273.0     1.8288   74.842680


,SEQN,Height(m),Weight(kg)
0,109266.0,1.6256,95.254320
1,109267.0,1.6002,58.966960
2,109268.0,1.6510,54.431040
3,109271.0,1.8288,100.697424
4,109273.0,1.8288,74.842680


In [178]:
df_weight_prep.dropna(inplace=True)
df_weight_prep.info()
df_weight_prep.to_csv('./data/Prep_data/prep_weight.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 18146 entries, 0 to 18695
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEQN        18146 non-null  float64
 1   Height(m)   18146 non-null  float64
 2   Weight(kg)  18146 non-null  float64
dtypes: float64(3)
memory usage: 567.1 KB


### HDL and cholesterol total

In [179]:
folder_path = "./data/raw_data/HDL"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)


Variable creada: HDL_L_2021
Variable creada: P_HDL_2017

Todas las variables creadas:
['HDL_L_2021', 'P_HDL_2017']


In [180]:
folder_path = "./data/raw_data/chol_total"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: P_TCHOL_2017
Variable creada: TCHOL_L_2021

Todas las variables creadas:
['P_TCHOL_2017', 'TCHOL_L_2021']


In [181]:
# Columnas a conservar
cols_deseadas = ['SEQN', 'LBDHDDSI', 'LBDHDLSI', 'LBDTCSI']

# Lista de nombres de DataFrames a trabajar manualmente
dfs_a_tratar = ['HDL_L_2021', 'TCHOL_L_2021', 'P_HDL_2017', 'P_TCHOL_2017']

# Filtrar columnas deseadas
for name in dfs_a_tratar:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ columnas deseadas no encontradas")

print("\n")

# Renombrar columnas
rename_dict = {
    'LBDHDDSI': 'HDL-Cholesterol(mmol/L)',
    'LBDHDLSI': 'HDL-Cholesterol(mmol/L)',
    'LBDTCSI': 'Total_Cholesterol(mmol/L)'
}

for name in dfs_a_tratar:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Merge manual de los dos pares de DataFrames
df_2021 = pd.merge(HDL_L_2021, TCHOL_L_2021, on='SEQN', how='outer')
df_2017 = pd.merge(P_HDL_2017, P_TCHOL_2017, on='SEQN', how='outer')

# Concatenar los dos resultados
df_HDL_TCHOL_prep = pd.concat([df_2021, df_2017], axis=0, ignore_index=True)

# Resultado final
print("\n🔍 DataFrame final HDL + TCHOL:")
df_HDL_TCHOL_prep.head()



HDL_L_2021: columnas filtradas → ['SEQN', 'LBDHDDSI']
TCHOL_L_2021: columnas filtradas → ['SEQN', 'LBDTCSI']
P_HDL_2017: columnas filtradas → ['SEQN', 'LBDHDDSI']
P_TCHOL_2017: columnas filtradas → ['SEQN', 'LBDTCSI']


HDL_L_2021: columnas renombradas → ['HDL-Cholesterol(mmol/L)']
TCHOL_L_2021: columnas renombradas → ['Total_Cholesterol(mmol/L)']
P_HDL_2017: columnas renombradas → ['HDL-Cholesterol(mmol/L)']
P_TCHOL_2017: columnas renombradas → ['Total_Cholesterol(mmol/L)']



🔍 DataFrame final HDL + TCHOL:


,SEQN,HDL-Cholesterol(mmol/L),Total_Cholesterol(mmol/L)
0,130378.0,1.16,6.83
1,130379.0,1.55,5.53
2,130380.0,1.27,4.84
3,130386.0,1.19,4.73
4,130387.0,1.09,5.25


In [182]:
df_HDL_TCHOL_prep.dropna(inplace=True)
df_HDL_TCHOL_prep.info()
df_HDL_TCHOL_prep.to_csv('./data/Prep_data/prep_HDL_TCHOL.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 17718 entries, 0 to 20265
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQN                       17718 non-null  float64
 1   HDL-Cholesterol(mmol/L)    17718 non-null  float64
 2   Total_Cholesterol(mmol/L)  17718 non-null  float64
dtypes: float64(3)
memory usage: 553.7 KB


### Glycohemoglobin

In [183]:
folder_path = "./data/raw_data/GHB"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: GHB_L_2021
Variable creada: P_GHB_2017

Todas las variables creadas:
['GHB_L_2021', 'P_GHB_2017']


In [184]:
# Columnas que quieres conservar (las que estén disponibles)
cols_deseadas = ["SEQN",'LBXGH']

for name in variable_names:
    df = globals()[name]
    
    # Filtrar columnas que existen en este DataFrame
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Diccionario de renombrado
rename_dict = {
    'LBXGH': 'Glycohemoglobin(%)'
}

# Renombrar columnas en todos los DataFrames
for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Ordenar todos los DataFrames por SEQN
for name in variable_names:
    df = globals()[name]
    if 'SEQN' in df.columns:
        df.sort_values('SEQN', inplace=True)
        df.reset_index(drop=True, inplace=True)
        globals()[name] = df

# Concatenar por filas (uno debajo de otro)
dfs_to_concat = [globals()[name] for name in variable_names]
df_GHB_prep = pd.concat(dfs_to_concat, axis=0, ignore_index=True)


# Mostrar resultado final
print("\nDataFrame final con unidades convertidas:")
print(df_GHB_prep.head())

# Guardar (opcional)
df_GHB_prep.head()

GHB_L_2021: columnas filtradas → ['SEQN', 'LBXGH']
P_GHB_2017: columnas filtradas → ['SEQN', 'LBXGH']


GHB_L_2021: columnas renombradas → ['Glycohemoglobin(%)']
P_GHB_2017: columnas renombradas → ['Glycohemoglobin(%)']



DataFrame final con unidades convertidas:
       SEQN  Glycohemoglobin(%)
0  130378.0                 5.6
1  130379.0                 5.6
2  130380.0                 6.2
3  130386.0                 5.1
4  130387.0                 5.9


,SEQN,Glycohemoglobin(%)
0,130378.0,5.6
1,130379.0,5.6
2,130380.0,6.2
3,130386.0,5.1
4,130387.0,5.9


In [185]:
df_GHB_prep.dropna(inplace=True)
df_GHB_prep.info()
df_GHB_prep.to_csv('./data/Prep_data/prep_GHB.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 16452 entries, 0 to 17607
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SEQN                16452 non-null  float64
 1   Glycohemoglobin(%)  16452 non-null  float64
dtypes: float64(2)
memory usage: 385.6 KB


### Insulin

In [186]:
folder_path = "./data/raw_data/INS"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: INS_L_2021
Variable creada: P_INS_2017

Todas las variables creadas:
['INS_L_2021', 'P_INS_2017']


In [187]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'LBDINSI', 'LB2INSI', 'LBXINSI']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'LBDINSI': 'Insulin (pmol/L)',
    'LB2INSI': 'Insulin (pmol/L)',
    'LBXINSI': 'Insulin (pmol/L)'
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if 'Insulin (pmol/L)' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_INS_prep = pd.concat(merged_dfs, axis=0, ignore_index=True)

# Resultado final
print("\n🔍 DataFrame final de insulina:")
print(df_INS_prep.head())


INS_L_2021: columnas filtradas → ['SEQN', 'LBDINSI']
P_INS_2017: columnas filtradas → ['SEQN', 'LBDINSI']


INS_L_2021: columnas renombradas → ['Insulin (pmol/L)']
P_INS_2017: columnas renombradas → ['Insulin (pmol/L)']


✅ INS_L_2021: añadido al DataFrame final
✅ P_INS_2017: añadido al DataFrame final

🔍 DataFrame final de insulina:
       SEQN  Insulin (pmol/L)
0  130378.0             93.18
1  130379.0            119.46
2  130380.0             97.98
3  130386.0             68.28
4  130394.0             43.20


In [188]:
df_INS_prep.dropna(inplace=True)
df_INS_prep.info()
df_INS_prep.to_csv('./data/Prep_data/prep_INS.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 8135 entries, 0 to 9085
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEQN              8135 non-null   float64
 1   Insulin (pmol/L)  8135 non-null   float64
dtypes: float64(2)
memory usage: 190.7 KB


### Diabetes (target)

In [189]:
folder_path = "./data/raw_data/diabetes"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: DIQ_L_2021
Variable creada: P_DIQ_2017

Todas las variables creadas:
['DIQ_L_2021', 'P_DIQ_2017']


In [190]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'DIQ010']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'DIQ010': 'Diabetes_diagnosticated',
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if 'Diabetes_diagnosticated' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_diabetes_prep = pd.concat( merged_dfs, axis=0, ignore_index=True)

# Resultado final
print("\n🔍 DataFrame final de insulina:")
print(df_diabetes_prep.head())


DIQ_L_2021: columnas filtradas → ['SEQN', 'DIQ010']
P_DIQ_2017: columnas filtradas → ['SEQN', 'DIQ010']


DIQ_L_2021: columnas renombradas → ['Diabetes_diagnosticated']
P_DIQ_2017: columnas renombradas → ['Diabetes_diagnosticated']


✅ DIQ_L_2021: añadido al DataFrame final
✅ P_DIQ_2017: añadido al DataFrame final

🔍 DataFrame final de insulina:
       SEQN  Diabetes_diagnosticated
0  130378.0                      2.0
1  130379.0                      2.0
2  130380.0                      1.0
3  130381.0                      2.0
4  130382.0                      2.0


In [191]:
df_diabetes_prep['Diabetes_diagnosticated'] = df_diabetes_prep['Diabetes_diagnosticated'].map({
    1.0: 1,  # Sí
    2.0: 0,  # No
    3.0: 1.0, # Limítrofe → se incluye como positivo
    9.0: np.nan  # No sabe
})

df_diabetes_prep.dropna(inplace=True)
df_diabetes_prep.info()
df_diabetes_prep.to_csv('./data/Prep_data/prep_diabetes.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 26714 entries, 0 to 26729
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SEQN                     26714 non-null  float64
 1   Diabetes_diagnosticated  26714 non-null  float64
dtypes: float64(2)
memory usage: 626.1 KB


### Blood pressure

In [192]:
folder_path = "./data/raw_data/BPQ"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: BPQ_L_2021
Variable creada: P_BPQ_2017

Todas las variables creadas:
['BPQ_L_2021', 'P_BPQ_2017']


In [193]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'BPQ020']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'BPQ020': 'High_blood_pressure?',
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if 'High_blood_pressure?' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_blood_prep = pd.concat( merged_dfs, axis=0, ignore_index=True)

df_blood_prep['High_blood_pressure?'] = df_blood_prep['High_blood_pressure?'].replace({9.0: np.nan})

# Resultado final
print("\n🔍 DataFrame final de insulina:")
df_blood_prep.head()


BPQ_L_2021: columnas filtradas → ['SEQN', 'BPQ020']
P_BPQ_2017: columnas filtradas → ['SEQN', 'BPQ020']


BPQ_L_2021: columnas renombradas → ['High_blood_pressure?']
P_BPQ_2017: columnas renombradas → ['High_blood_pressure?']


✅ BPQ_L_2021: añadido al DataFrame final
✅ P_BPQ_2017: añadido al DataFrame final

🔍 DataFrame final de insulina:


,SEQN,High_blood_pressure?
0,130378.0,1.0
1,130379.0,1.0
2,130380.0,2.0
3,130384.0,2.0
4,130385.0,2.0


In [194]:
df_blood_prep.dropna(inplace=True)
df_blood_prep.info()
df_blood_prep.to_csv('./data/Prep_data/prep_blood.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 18671 entries, 0 to 18695
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SEQN                  18671 non-null  float64
 1   High_blood_pressure?  18671 non-null  float64
dtypes: float64(2)
memory usage: 437.6 KB


### Smoking use

In [195]:
folder_path = "./data/raw_data/SMQ"
variable_names = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and ("_L_" in filename or "P_" in filename):
        file_path = os.path.join(folder_path, filename)
        var_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_")
        globals()[var_name] = pd.read_csv(file_path)
        variable_names.append(var_name)
        print(f"Variable creada: {var_name}")

print("\nTodas las variables creadas:")
print(variable_names)

Variable creada: P_SMQ_2017
Variable creada: SMQ_L_2021

Todas las variables creadas:
['P_SMQ_2017', 'SMQ_L_2021']


In [196]:
# Columnas a conservar (solo insulina)
cols_deseadas = ['SEQN', 'SMQ020']

# Filtrar columnas deseadas en cada DataFrame
for name in variable_names:
    df = globals()[name]
    cols_existentes = [col for col in cols_deseadas if col in df.columns]
    
    if cols_existentes:
        df = df[cols_existentes]
        globals()[name] = df
        print(f"{name}: columnas filtradas → {cols_existentes}")
    else:
        print(f"{name}: ⚠️ ninguna columna deseada encontrada")

print("\n")

# Renombrar columnas de insulina (si existen)
rename_dict = {
    'SMQ020': '100_cigarrettes_life?'
}

for name in variable_names:
    df = globals()[name]
    cols_to_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
    df.rename(columns=cols_to_rename, inplace=True)
    globals()[name] = df
    print(f"{name}: columnas renombradas → {list(cols_to_rename.values())}")

print("\n")

# Concatenar todos los DataFrames de insulina (sin importar año)
merged_dfs = []

for name in variable_names:
    df = globals()[name]
    if '100_cigarrettes_life?' in df.columns:
        merged_dfs.append(df)
        print(f"✅ {name}: añadido al DataFrame final")
    else:
        print(f"⚠️ {name}: no contiene columna de insulina")

# Concatenar todos los DataFrames de insulina
df_smoking_prep = pd.concat( merged_dfs, axis=0, ignore_index=True)

df_smoking_prep['100_cigarrettes_life?'] = df_smoking_prep['100_cigarrettes_life?'].replace({9.0: np.nan, 7.0: np.nan})

# Resultado final
print("\n🔍 DataFrame final de insulina:")
df_smoking_prep.head()

P_SMQ_2017: columnas filtradas → ['SEQN', 'SMQ020']
SMQ_L_2021: columnas filtradas → ['SEQN', 'SMQ020']


P_SMQ_2017: columnas renombradas → ['100_cigarrettes_life?']
SMQ_L_2021: columnas renombradas → ['100_cigarrettes_life?']


✅ P_SMQ_2017: añadido al DataFrame final
✅ SMQ_L_2021: añadido al DataFrame final

🔍 DataFrame final de insulina:


,SEQN,100_cigarrettes_life?
0,109264.0,NaN
1,109266.0,2.0
2,109267.0,2.0
3,109268.0,2.0
4,109271.0,1.0


In [197]:
df_smoking_prep.dropna(inplace=True)
df_smoking_prep.info()
df_smoking_prep.to_csv('./data/Prep_data/prep_smoking.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 17809 entries, 1 to 20151
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SEQN                   17809 non-null  float64
 1   100_cigarrettes_life?  17809 non-null  float64
dtypes: float64(2)
memory usage: 417.4 KB


### MERGE ALL DATAFRAMES TO PREDICT

In [198]:
df_final_prep = pd.merge(pre_demographic, df_weight_prep, on="SEQN", how="inner")
df_final_prep = pd.merge(df_final_prep, df_HDL_TCHOL_prep, on="SEQN", how="inner")
df_final_prep = pd.merge(df_final_prep, df_GHB_prep, on="SEQN", how="inner")
df_final_prep = pd.merge(df_final_prep, df_INS_prep, on="SEQN", how="inner")
df_final_prep = pd.merge(df_final_prep, df_blood_prep, on="SEQN", how="inner")
df_final_prep = pd.merge(df_final_prep, df_smoking_prep, on="SEQN", how="inner") 
df_final_prep = pd.merge(df_final_prep, df_diabetes_prep, on="SEQN", how="inner")  
df_final_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6123 entries, 0 to 6122
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQN                       6123 non-null   float64
 1   Gender                     6123 non-null   float64
 2   Age(year)                  6123 non-null   float64
 3   Race                       6123 non-null   float64
 4   Country_of_birth           6123 non-null   float64
 5   Income_category            6123 non-null   float64
 6   Height(m)                  6123 non-null   float64
 7   Weight(kg)                 6123 non-null   float64
 8   HDL-Cholesterol(mmol/L)    6123 non-null   float64
 9   Total_Cholesterol(mmol/L)  6123 non-null   float64
 10  Glycohemoglobin(%)         6123 non-null   float64
 11  Insulin (pmol/L)           6123 non-null   float64
 12  High_blood_pressure?       6123 non-null   float64
 13  100_cigarrettes_life?      6123 non-null   float

In [199]:
df_final_prep.head()

,SEQN,Gender,Age(year),Race,Country_of_birth,Income_category,Height(m),Weight(kg),HDL-Cholesterol(mmol/L),Total_Cholesterol(mmol/L),Glycohemoglobin(%),Insulin (pmol/L),High_blood_pressure?,100_cigarrettes_life?,Diabetes_diagnosticated
0,130378.0,1.0,43.0,5.0,2.0,5.0,1.8034,86.182480,1.16,6.83,5.6,93.18,1.0,1.0,0.0
1,130379.0,1.0,66.0,3.0,1.0,5.0,1.7780,99.790240,1.55,5.53,5.6,119.46,1.0,1.0,0.0
2,130380.0,2.0,44.0,2.0,2.0,2.0,1.5240,68.038800,1.27,4.84,6.2,97.98,2.0,2.0,1.0
3,130386.0,1.0,34.0,1.0,1.0,2.0,1.7272,90.718400,1.19,4.73,5.1,68.28,2.0,1.0,0.0
4,130394.0,1.0,51.0,3.0,1.0,5.0,1.7780,78.925008,1.24,4.73,4.8,43.20,2.0,2.0,0.0


In [200]:
df_final_prep.to_csv('./data/Prep_data/prep_data.csv', index=False)